<a href="https://colab.research.google.com/github/FadilSeputra/braisee-machine-learning/blob/main/offline_road.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessing Gambar

In [ ]:
def preprocess_image_mobile(image):
    # Resize gambar dari kamera
    height, width = image.shape[:2]
    if max(height, width) > 640:  # Maksimal resolusi
        scale = 640 / max(height, width)
        image = cv2.resize(image, (int(width * scale), int(height * scale)))
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    _, binary_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    clean_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    contrast_image = cv2.convertScaleAbs(clean_image, alpha=2, beta=0)  # Penajaman kontras
    normalized_image = image / 255.0  # Skala nilai pixel ke rentang [0, 1]

    return normalized_image, binary_image  # Return both images


# Algoritma untuk Sliding Window dan Deteksi Bounding Box


In [ ]:
# def sliding_window_mobile(image, step_size, initial_box_size, max_box_size, model,max_iterations=50):
#     height, width = image.shape
#     detected_boxes = []
#     predictions = []
#     box_size = initial_box_size
#     iteration = 0

#     while box_size <= max_box_size and iteration < max_iterations:
#         for y in range(0, height - box_size + 1, step_size):
#             for x in range(0, width - box_size + 1, step_size):
#                 roi = image[y:y + box_size, x:x + box_size]
#                 roi_resized = cv2.resize(roi, (28, 28))  # Ubah ukuran hasil ekstrasi agar sesuai model cnn yang telah dibuat
#                 roi_normalized = roi_resized / 255.0
#                 roi_input = np.expand_dims(roi_normalized, axis=(0, -1))

#                 prediction = model.predict(roi_input, verbose=0)
#                 predicted_class = np.argmax(prediction)
#                 confidence = np.max(prediction)

#                 if predicted_class !=26 and confidence > 0.5:  # 0 adalah "tidak ada huruf"
#                     detected_boxes.append((x, y, x + box_size, y + box_size))
#                     predictions.append((predicted_class, confidence))

#         if predictions:
#             break
#         box_size += step_size  # memperbesar bounding box
#         iteration += 1

#     return detected_boxes, predictions

def sliding_window_mobile(image, step_size, initial_box_size, max_box_size, model, max_iterations=50):
    height, width = image.shape
    detected_boxes = []
    predictions = []
    box_size = initial_box_size
    iteration = 0

    while box_size <= max_box_size and iteration < max_iterations:
        for y in range(0, height - box_size + 1, step_size):
            for x in range(0, width - box_size + 1, step_size):
                roi = image[y:y + box_size, x:x + box_size]
                roi_resized = cv2.resize(roi, (28, 28))  # Adjust size to match model input
                roi_normalized = roi_resized / 255.0  # Normalize image to [0, 1]

                # Convert to INT8 format: scale and shift values to [-128, 127] if model expects INT8
                roi_int8 = np.round(roi_normalized * 255).astype(np.int8)  # Convert to INT8

                # Ensure the shape is (1, 28, 28, 1) for the model
                roi_input = np.expand_dims(roi_int8, axis=(0, -1))

                # Get input and output details
                input_details = model.get_input_details()
                output_details = model.get_output_details()

                # Set input tensor
                model.set_tensor(input_details[0]['index'], roi_input)

                # Run inference
                model.invoke()

                # Get the output tensor
                prediction = model.get_tensor(output_details[0]['index'])
                predicted_class = np.argmax(prediction)
                confidence = np.max(prediction)

                if predicted_class != 26 and confidence > 0.5:  # Class 26 is "empty"
                    detected_boxes.append((x, y, x + box_size, y + box_size))
                    predictions.append((predicted_class, confidence))

        if predictions:
            break

        box_size += step_size  # Increase box size for the next iteration
        iteration += 1

    return detected_boxes, predictions



# penyusunan gambar


In [ ]:
def construct_sentence_mobile(bounding_boxes, predictions, class_mapping):
    # Sort bounding boxes
    sorted_indices = sorted(range(len(bounding_boxes)), key=lambda i: (bounding_boxes[i][1], bounding_boxes[i][0]))
    sentence = []
    for i in sorted_indices:
        pred_class = predictions[i][0]
        if pred_class in class_mapping:
            sentence.append(class_mapping[pred_class])
        else:
            print(f"Warning: Predicted class {pred_class} not found in class_mapping.")

    # Return the constructed sentence
    return ''.join(sentence)



# main pipeline

In [ ]:
def main_pipeline_mobile(image_path, tflite_model_path, class_mapping):
    # Preprocessing
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not loaded. Check the image path.")
        return

    image, thresh = preprocess_image_mobile(image)

    # Load model
    try:
        model = tf.lite.Interpreter(model_path=tflite_model_path)
        model.allocate_tensors()
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Sliding window
    detected_boxes, predictions = sliding_window_mobile(
        thresh, step_size=2, initial_box_size=20, max_box_size=128, model=model
    )
    print(f"Detected boxes: {detected_boxes}")
    print(f"Predictions: {predictions}")

    # Construct sentence from detected boxes and predictions
    sentence = construct_sentence_mobile(detected_boxes, predictions, class_mapping)
    print(f"Constructed sentence: {sentence}")

    # Visualize result (overlay sentence and boxes on image)
    display_result_mobile(image, sentence)



# class mapping

In [ ]:
# model label
map={'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8,
                  'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17,
                  's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, 'empty': 26}
model_path="/content/drive/MyDrive/Braille_image_classifier_1_fix_quantized.tflite"
image ="/content/drive/MyDrive/images/train/0000000.jpg"
main_pipeline_mobile(image,model_path,map)

Detected boxes: [(80, 34, 100, 54), (82, 34, 102, 54), (84, 34, 104, 54), (86, 34, 106, 54), (88, 34, 108, 54), (90, 34, 110, 54), (92, 34, 112, 54), (94, 34, 114, 54), (96, 34, 116, 54), (98, 34, 118, 54), (100, 34, 120, 54), (102, 34, 122, 54), (106, 34, 126, 54), (220, 42, 240, 62), (76, 50, 96, 70), (80, 50, 100, 70), (54, 246, 74, 266), (62, 248, 82, 268), (76, 250, 96, 270), (78, 250, 98, 270), (80, 250, 100, 270), (148, 250, 168, 270), (92, 262, 112, 282)]
Predictions: [(10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (10, 16), (0, 59), (0, 26), (10, 83), (10, 92), (10, 101), (10, 49), (10, 63), (10, 47), (4, 36)]
Constructed sentence: 
Detected sentence: 


# model doc

In [ ]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
def load_and_label_image(image_path, label):
    img = load_img(image_path, target_size=(28, 28), color_mode='grayscale')
    img_array = img_to_array(img) / 255.0  # Normalisasi gambar
    return img_array, label

def generate_labels_from_filenames(image_dir):
    images = []
    labels = []
    label_dict = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8,
                  'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17,
                  's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, 'empty': 26}

    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)

        # Jika file adalah gambar kosong (misalnya, "kosong.jpg" atau gambar dengan label "empty")
        if filename.startswith("empty"):
            label = label_dict['empty']
        elif filename.endswith('.jpg'):
            label = label_dict[filename[0]]  # Mengambil huruf pertama sebagai label
        else:
            continue

        img_array, label = load_and_label_image(image_path, label)
        images.append(img_array)
        labels.append(label)

    return np.array(images), to_categorical(labels)

# Contoh penggunaan
image_dir = '/content/drive/MyDrive/Braille_Dataset'#my google drive data set
X, y = generate_labels_from_filenames(image_dir)

In [ ]:
# split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verifikasi pembagian
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

In [ ]:
#model
model = keras.Sequential([
    # convo, pool dan norm pertama
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', strides=(1, 1), activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),

    # convo, pool dan norm kedua
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.2, input_shape=(28,1)),
    keras.layers.BatchNormalization(),

    # convo, pool dan norm ketiga
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', strides=(1,1), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.25, input_shape=(28,1)),
    keras.layers.BatchNormalization(),

    #flatten
    keras.layers.Flatten(),

    #dense layer to connect and decress overfitting
    keras.layers.Dense(units=512, activation="relu"),
    keras.layers.Dropout(0.5, input_shape=(28,1)),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(units=288, activation="relu"),
    keras.layers.Dense(units=27, activation="softmax")])

In [ ]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(x=X_train,
                    y=y_train,
                    epochs=100,)

In [ ]:
test_loss, test_acc = model.evaluate(X_train, y_train)

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

def sliding_window_with_classification(image, cnn_model, step_size=5, initial_box_size=20, max_box_size=100):
    height, width = image.shape
    detected_characters = []
    box_size = initial_box_size
    extracted_boxes = []

    while box_size <= max_box_size:
        has_detection = False
        for y in range(0, height - box_size + 1, step_size):
            for x in range(0, width - box_size + 1, step_size):
                roi = image[y:y + box_size, x:x + box_size]
                roi_resized = cv2.resize(roi, (28, 28))
                roi_normalized = roi_resized / 255.0
                roi_input = np.expand_dims(roi_normalized, axis=(0, -1))
                prediction = cnn_model.predict(roi_input, verbose=0)
                predicted_class = np.argmax(prediction)
                confidence = np.max(prediction)

                if predicted_class != 26 and confidence > 0.5:
                    has_detection = True
                    detected_characters.append((predicted_class, confidence, (x, y, x + box_size, y + box_size)))
                    extracted_boxes.append((roi_resized, (x, y, x + box_size, y + box_size)))

        if has_detection:
            break
        box_size += step_size

    return detected_characters, extracted_boxes
def arrange_characters_to_sentence(detected_characters):
    detected_characters.sort(key=lambda x: (x[2][1], x[2][0]))
    sentence = ''.join([chr(65 + char[0]) for char in detected_characters])
    return sentence
def process_braille_image(image_path, cnn_model_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Gambar tidak ditemukan")
    cnn_model = tf.keras.models.load_model(cnn_model_path)
    detected_characters, extracted_boxes = sliding_window_with_classification(
        image, cnn_model, step_size=10, initial_box_size=20, max_box_size=100
    )
    sentence = arrange_characters_to_sentence(detected_characters)
    print("Kalimat yang terdeteksi:", sentence)
    for _, _, (x1, y1, x2, y2) in detected_characters:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow("Detected Bounding Boxes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return sentence
